In [1]:
import pandas as pd
import numpy as np
import csv
import jieba
import random
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from tqdm import tqdm

In [2]:
branddata = pd.read_csv('整理的品牌資料集/cleaned_brand_train.csv')
branddata = branddata[['clean_name', 'final_brand']]

In [3]:
branddata

,clean_name,final_brand
0,元山熱水瓶ys540ap,元山
1,姍拉娜治痘洗面乳,姍拉娜
2,台鹽海洋鹼性離子水,台鹽
3,妮傲絲翠果酸深層保養乳液,neo-tec妮傲絲翠
4,舒潔棉柔舒適迪士尼抽取式衛生紙,舒潔
...,...,...
70034,惠而浦瓦斯型滾筒蒸氣可堆疊乾衣機8twgd6622hw,whirlpool惠而浦
70035,康乃馨成人紙尿褲l,康乃馨
70036,日本芮芙茹零矽靈洗髮露頭皮保養,reveur芮芙茹
70037,crest長效鎖白牙膏輕柔鑽白,crest


In [4]:
branddata['cutname'] = ''
for i in range(len(branddata)):
    branddata['cutname'][i] = ' '.join(jieba.cut_for_search(branddata['clean_name'][i]))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/jieba.cache
Loading model cost 0.441 seconds.
Prefix dict has been built successfully.


In [5]:
branddata

,clean_name,final_brand,cutname
0,元山熱水瓶ys540ap,元山,元山 熱 水瓶 ys540ap
1,姍拉娜治痘洗面乳,姍拉娜,姍拉娜治痘 洗面 洗面乳
2,台鹽海洋鹼性離子水,台鹽,台鹽 海洋 鹼性 離子 水
3,妮傲絲翠果酸深層保養乳液,neo-tec妮傲絲翠,妮 傲絲翠 果酸 深層 保養 乳液
4,舒潔棉柔舒適迪士尼抽取式衛生紙,舒潔,舒潔 棉柔 舒適 迪士尼 抽取 式 衛生紙
...,...,...,...
70034,惠而浦瓦斯型滾筒蒸氣可堆疊乾衣機8twgd6622hw,whirlpool惠而浦,惠而浦 瓦斯 型 滾筒 蒸氣 可堆 疊 乾衣機 8twgd6622hw
70035,康乃馨成人紙尿褲l,康乃馨,康乃馨 成人 紙尿褲 l
70036,日本芮芙茹零矽靈洗髮露頭皮保養,reveur芮芙茹,日本 芮芙茹 零 矽靈 洗 髮 露頭 皮 保養
70037,crest長效鎖白牙膏輕柔鑽白,crest,crest 長 效鎖白 牙膏 輕柔 鑽白


In [6]:
def Splitdata(data, ratio=0.8):
    train_x = []
    train_y = []
    test_x = []
    test_y = []
    for uniname in data['final_brand'].unique():
        matchdata = data[data['final_brand'] == uniname]
        random.seed(1)
        matchdata = shuffle(matchdata)
        train_num = round(0.8*len(matchdata))
        
        for i in matchdata['cutname'][:train_num]:
            train_x.append(i)
        for i in matchdata['final_brand'][:train_num]:
            train_y.append(i)
        
        for i in matchdata['cutname'][train_num:]:
            test_x.append(i)
        for i in matchdata['final_brand'][train_num:]:
            test_y.append(i)
    
    return train_x, train_y, test_x, test_y

In [7]:
x_train, y_train, x_test, y_test = Splitdata(branddata)

In [8]:
model = make_pipeline(TfidfVectorizer(), MultinomialNB(alpha = 0.01))

In [9]:
model.fit(x_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB(alpha=0.01))])

In [10]:
model.score(x_test, y_test)

0.9016083254493851

In [12]:
y_pred = model.predict(x_test)
f1_score(y_test,y_pred,average='macro')

0.6907371245165372

In [13]:
def predict_brand(s, train=x_train, model=model):
    pred = model.predict([s])
    return pred[0]

In [14]:
prddata = pd.read_csv('整理的品牌資料集/cleaned_brand_test.csv')
prddata = prddata[['name', 'clean_name','brand']]
prddata['brand'] = ''

In [16]:
prddata['cutname'] = ''
for i in range(len(prddata)):
    prddata['cutname'][i] = ' '.join(jieba.cut_for_search(prddata['clean_name'][i]))

In [19]:
prddata

,name,clean_name,brand,cutname
0,【heme 喜蜜】輕透柔光粉底液 spf50+ ★★★ - 30ml(2入組),heme 喜蜜輕透柔光粉底液 spf50,mkup美咖,heme 喜蜜 輕透 柔光 粉底 粉底液 spf50
1,【肌研】卵肌溫和去角質化粧水170ml 2入,肌研卵肌溫和去角質化粧水,rohto樂敦,肌研卵 肌溫 和 去 角質化 粧 水
2,"c.m康媚絲洗髮精,強化",cm康媚絲洗髮精強化,reebok,cm 康媚絲 洗 髮 精強化
3,br超柔潤保濕面膜 5片,br超柔潤保濕面膜,za,br 超柔潤 保濕 面膜
4,(即期品)(買1送1)dejavu 睫采分明立體睫毛膏_黑色3.1g(效期2022/6/30),即期品dejavu 睫采分明立體睫毛膏黑色,dejavu,即期 品 dejavu 睫采 分明 立體 睫毛 睫毛膏 黑色
...,...,...,...,...
97154,石澤研究所毛穴撫子角質對策泡洗顏100g,石澤研究所毛穴撫子角質對策泡洗顏,,石澤 研究 研究所 毛穴 撫子 角質 對策 泡 洗顏
97155,金牌台灣啤酒330ml6入,金牌台灣啤酒,,金牌 台灣 啤酒
97156,日清 中杯麵 醬油,日清 中杯麵 醬油,,日清 中杯 麵 醬油
97157,杜蕾斯活力裝衛生套３個／盒,杜蕾斯活力裝衛生套３個／盒,,杜蕾斯 活力 裝衛生 套 ３ 個 ／ 盒


In [26]:
for i in tqdm(range(len(prddata))):
    if not prddata['cutname'][i].isnumeric():
        prddata['brand'][i] = predict_brand(prddata['cutname'][i])
    else:
        prddata['brand'][i] = prddata['cutname'][i]

100%|██████████████████████████████████| 97159/97159 [19:52:36<00:00,  1.36it/s]


In [27]:
prddata

,name,clean_name,brand,cutname
0,【heme 喜蜜】輕透柔光粉底液 spf50+ ★★★ - 30ml(2入組),heme 喜蜜輕透柔光粉底液 spf50,mkup美咖,heme 喜蜜 輕透 柔光 粉底 粉底液 spf50
1,【肌研】卵肌溫和去角質化粧水170ml 2入,肌研卵肌溫和去角質化粧水,rohto樂敦,肌研卵 肌溫 和 去 角質化 粧 水
2,"c.m康媚絲洗髮精,強化",cm康媚絲洗髮精強化,reebok,cm 康媚絲 洗 髮 精強化
3,br超柔潤保濕面膜 5片,br超柔潤保濕面膜,za,br 超柔潤 保濕 面膜
4,(即期品)(買1送1)dejavu 睫采分明立體睫毛膏_黑色3.1g(效期2022/6/30),即期品dejavu 睫采分明立體睫毛膏黑色,dejavu,即期 品 dejavu 睫采 分明 立體 睫毛 睫毛膏 黑色
...,...,...,...,...
97154,石澤研究所毛穴撫子角質對策泡洗顏100g,石澤研究所毛穴撫子角質對策泡洗顏,tsaio上山採藥,石澤 研究 研究所 毛穴 撫子 角質 對策 泡 洗顏
97155,金牌台灣啤酒330ml6入,金牌台灣啤酒,台啤,金牌 台灣 啤酒
97156,日清 中杯麵 醬油,日清 中杯麵 醬油,nissin日清,日清 中杯 麵 醬油
97157,杜蕾斯活力裝衛生套３個／盒,杜蕾斯活力裝衛生套３個／盒,mentholatum曼秀雷敦,杜蕾斯 活力 裝衛生 套 ３ 個 ／ 盒


In [29]:
df = prddata[['name', 'brand']]
df.to_csv('brand_test(answer).csv', encoding="utf-8")

In [30]:
df.to_csv('brand_test(answer)sig.csv', encoding="utf-8-sig")